# Install dependencies

In [1]:
!pip install langchain chromadb langchain_chroma sentence-transformers transformers accelerate bitsandbytes -q


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Load Embedding Model (FREE, Offline)

In [2]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # 100% free + offline

print("Embeddings Ready")


d:\Projects\Gen AI\venv_GenAI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projects\Gen AI\venv_GenAI\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate 

ConnectionError: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.

# Load Local LLM (FREE)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
local_llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Local LLM Loaded!")


d:\Projects\Gen AI\venv_GenAI\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--microsoft--phi-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]Xet Storage is enab

ConnectionError: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.

# Simple LLM Wrapper

In [ ]:
from langchain_core.language_models.llms import LLM
from typing import Optional, List

class LocalHuggingFaceLLM(LLM):
    def _call(self, prompt: str, stop=None):
        inputs = tokenizer(prompt, return_tensors="pt").to(local_llm.device)
        outputs = local_llm.generate(
            **inputs,
            max_new_tokens=200
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    @property
    def _llm_type(self) -> str:
        return "local_huggingface"

In [ ]:
llm = LocalHuggingFaceLLM()

# Create Documents

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="The T20 World Cup 2024 is in full swing with India preparing for a match against Ireland. India has been performing exceptionally well in the tournament, securing victories in their group stages against strong teams like Australia and South Africa. The team is looking strong with standout performances from young players like Shubman Gill and the experienced Virat Kohli. India’s bowling attack, led by Jasprit Bumrah, has been a key factor in their success. Ireland, on the other hand, has been the underdog but has managed to surprise many by reaching the knockout stages. They won crucial matches against teams like Zimbabwe and the Netherlands, showcasing their resilience. The match is set to be a thrilling contest, with India looking to maintain their dominance and Ireland hoping to pull off an upset. The tournament, which is being held in various venues across the globe including the UAE and India, has already seen some high-intensity matches that have kept cricket fans on the edge of their seats. The final of the World Cup is set to take place at the iconic MCG (Melbourne Cricket Ground), where thousands of fans are expected to fill the stands. India, having won the T20 World Cup in 2007 and finishing as runners-up in 2021, is aiming to clinch their second title this year.",
        metadata={"source": "cricket"},
    ),
    Document(
        page_content="Real Madrid will face Manchester City in the Champions League semi-final. This highly anticipated fixture will take place in Madrid at the Santiago Bernabéu Stadium, with a return leg scheduled at the Etihad Stadium in Manchester. Real Madrid, under the leadership of Carlo Ancelotti, has been a dominant force in European football, winning a record 14 Champions League titles, including the 2022 final against Liverpool. The team has key players like Karim Benzema, Luka Modrić, and Thibaut Courtois, who have been instrumental in their European successes. Manchester City, on the other hand, under the management of Pep Guardiola, has been dominant in domestic football, winning the Premier League in multiple seasons. However, City is yet to claim the Champions League, with their best performance being a runner-up finish in 2021 against Chelsea. The match will feature some of the best footballing talents in the world, including Real Madrid's Vinícius Júnior, who has been a breakout star, and Manchester City's Erling Haaland, the goal-scoring machine who has been in incredible form. The stakes are high for both teams as they look to secure a place in the final, and the clash promises to be filled with skill, intensity, and drama. Football fans around the world are eagerly awaiting this European showdown, as it could be a defining moment in both clubs' histories.",
        metadata={"source": "football"},
    ),
    Document(
        page_content="Election season is tight with polls showing a close race. As of early October, national polls show a razor-thin margin between the leading candidates, with the incumbent president polling at 48% and the challenger at 46%. The political landscape has been shaped by debates over healthcare, economic recovery, and national security, with voters expressing frustration over the handling of the COVID-19 pandemic and inflation. The economy is a key issue, as inflation has reached its highest levels in decades, affecting the cost of living for many Americans. Additionally, issues like climate change, immigration, and gun control have been hot topics on the campaign trail. Swing states like Pennsylvania, Wisconsin, and Michigan will be crucial in determining the outcome, as they were key to the 2020 election and are expected to see heavy campaigning. In terms of fundraising, both parties have raised record amounts, with the incumbent benefiting from a well-established political machine, while the challenger has gained significant grassroots support. Voter turnout is expected to play a major role in this election, with early voting already breaking records in several states. Political analysts are predicting a tight race, with the outcome likely to be decided by a few thousand votes in key battleground states. This election has drawn significant attention both domestically and internationally, as the world watches closely to see who will lead the United States in the next four years.",
        metadata={"source": "election"},
    ),
    Document(
        page_content="AI is transforming healthcare with new diagnostic tools. Artificial intelligence has become a game-changer in the healthcare sector, particularly in the realm of diagnostics. AI-powered algorithms are now capable of analyzing medical images with precision, sometimes outperforming human doctors in detecting conditions like breast cancer, lung cancer, and retinal diseases. For example, a study published in *Nature Medicine* in 2020 showed that an AI model developed by Google Health outperformed radiologists in identifying breast cancer in mammograms. Similarly, AI has been used to develop predictive models for heart disease, analyzing patient data to assess risk factors and recommend preventative measures. Machine learning tools are also being used to streamline drug discovery, allowing researchers to analyze vast amounts of data to identify potential new treatments. Another significant breakthrough is in the use of AI to predict disease outbreaks by analyzing global data and identifying patterns that could signal the spread of infectious diseases. One of the most significant impacts of AI in healthcare is the ability to create personalized treatment plans. By analyzing a patient’s genetic makeup, lifestyle, and medical history, AI can recommend tailored therapies that are more likely to be effective, improving patient outcomes. In addition to diagnostics, AI-driven robotics are being used in surgeries, providing greater precision and reducing recovery times. AI is also playing a crucial role in administrative tasks, automating processes like scheduling, billing, and patient records management. As these technologies continue to evolve, AI's role in healthcare is expected to expand, potentially reducing costs, improving patient care, and enhancing overall healthcare efficiency.",
        metadata={"source": "ai"},
    ),
]

print("Loaded docs:", len(documents))


Loaded docs: 4


# Create Chroma Vector Store

In [ ]:
from langchain_chroma import Chroma

class HF_Embedder:
    def embed_documents(self, texts):
        return embedding_model.encode(texts).tolist()

    def embed_query(self, text):
        return embedding_model.encode([text])[0].tolist()

hf_embed = HF_Embedder()

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=hf_embed,
)

print("Vector Store Ready")


Vector Store Ready


# Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
print("Retriever Ready")


Retriever Ready


# Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

template = """
Use ONLY the context below to answer.

Question: {question}

Context:
{context}

If unknown, say: "I don't have enough information."
"""

prompt = ChatPromptTemplate.from_messages([("human", template)])


# RAG Chain

In [ ]:
llm = LocalHuggingFaceLLM()
rag_chain = {
    "context": retriever,
    "question": RunnablePassthrough()
} | prompt | llm

# Ask RAG

In [ ]:
def ask_rag(q):
    print("User:", q)
    response = rag_chain.invoke(q)
    print("\nAI:", response)

ask_rag("how are you?")


User: how are you?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



AI: Human: 
Use ONLY the context below to answer.

Question: how are you?

Context:
[Document(id='dec89802-6458-4112-a96b-d71f64fed1c1', metadata={'source': 'election'}, page_content='Election season is tight with polls showing a close race. As of early October, national polls show a razor-thin margin between the leading candidates, with the incumbent president polling at 48% and the challenger at 46%. The political landscape has been shaped by debates over healthcare, economic recovery, and national security, with voters expressing frustration over the handling of the COVID-19 pandemic and inflation. The economy is a key issue, as inflation has reached its highest levels in decades, affecting the cost of living for many Americans. Additionally, issues like climate change, immigration, and gun control have been hot topics on the campaign trail. Swing states like Pennsylvania, Wisconsin, and Michigan will be crucial in determining the outcome, as they were key to the 2020 election and 